In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
%matplotlib inline
pd.set_option('display.max_columns', 500)
print("Seaborn version: ", sns.__version__)

Seaborn version:  0.9.0


#### Power Conference Feature

From EDA we know that a small number of conferences account for 70% of all confernece wins. 

If one of the two teams is in a power conference, indicate the advantage that team has for winning the game.

In [2]:
# Compare the conferences of two teams playing in a game.
# If both teams are in power conference or both teams are not in a power conference return 0.
# If the opponent team is in a power conference and the first team is not in a power conference return -1
# If the first team is a power conference team and opponent is not then return 1.

def conf_compare(team_conf, opp_conf):
    team_top = team_conf in top_tournament_conferences_list
    opp_top = opp_conf in top_tournament_conferences_list
    if team_top == opp_top:
        return 0
    elif team_top :
        return 1
    else:
        return -1

In [3]:
def join_feature_name_with_importance_value(features, importances):
    """
    Join via a list of tuples, feature names with their importance values
    :param features: data frame whose features are represented by columns used by classifier
    :param importances: feature importance scores assigned by classifier
    :return: sorted list (highest importances first) of feature,importance tuples
    """
    if features.columns.shape[0] != importances.shape[0]:
        return []

    feature_importances = []
    for item in range(features.columns.shape[0]):
        feature_importances.append((features.columns[item], importances[item]))
    feature_importances_sorted = sorted(feature_importances, reverse=True, key=lambda kv: abs(kv[1]))

    return feature_importances_sorted


In [4]:
def display_important_features_regression(classifier, features):
    importances = classifier.coef_[0]
    #print("Importances: ", importances) 
    feature_importances = join_feature_name_with_importance_value(features, importances)
    print('Coefficient Values')
    for items in feature_importances:
        print(items[0]," ", items[1])

In [5]:
def get_tournament_record(df, season, round_, team):
    return df[(df['season_t'] == season) & (df['round']== round_) & (df['team']==team)]


In [6]:
def print_game_info(df, season, round_, team):
    t_rec = get_tournament_record(df, season, round_, team)
    opp_team = t_rec['opp_team'].iloc[0]
    print('\t\t',team,'\t',opp_team)
    print('Seed','\t\t',"%6d"%t_rec['team_seed'].iloc[0],'\t',"%6d"%t_rec['opp_team_seed'].iloc[0])
    print('PPG','\t\t',"%6.2f"%t_rec['pts_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['pts_avg_o'].iloc[0])
    print('Allow PPG','\t',"%6.2f"%t_rec['opp_pts_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['opp_pts_avg_o'].iloc[0])
    print('Poss Avg','\t',"%6.2f"%t_rec['poss_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['poss_avg_o'].iloc[0])
    print('FG %','\t\t',"%6.2f"%(100*t_rec['fg_pct_t'].iloc[0]),'\t',"%6.2f"%(100*t_rec['fg_pct_o'].iloc[0]))
    print('Allow FG %','\t',"%6.2f"%(100*t_rec['allow_fg_pct_t'].iloc[0]),'\t',"%6.2f"%(100*t_rec['allow_fg_pct_o'].iloc[0]))
    print('Off Rb Avg','\t',"%6.2f"%t_rec['off_rebs_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['off_rebs_avg_o'].iloc[0])
    print('Allow Off Rb Avg',"%6.2f"%t_rec['allow_off_rebs_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['allow_off_rebs_avg_o'].iloc[0])
    print('Def Rb Avg','\t',"%6.2f"%t_rec['def_rebs_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['def_rebs_avg_o'].iloc[0])
    print('Allow Def Rb Avg',"%6.2f"%t_rec['allow_def_rebs_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['allow_def_rebs_avg_o'].iloc[0])  
    print('FT ATT Avg','\t',"%6.2f"%t_rec['ft_att_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['ft_att_avg_o'].iloc[0]) 
    print('Allow FT ATT Avg',"%6.2f"%t_rec['allow_ft_att_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['allow_ft_att_avg_o'].iloc[0])
    print('FT %','\t\t',"%6.2f"%(100*t_rec['ft_pct_t'].iloc[0]),'\t',"%6.2f"%(100*t_rec['ft_pct_o'].iloc[0]))
    print('Turnover Avg','\t',"%6.2f"%t_rec['to_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['to_avg_o'].iloc[0])
    print('Takeaway Avg','\t',"%6.2f"%t_rec['steal_avg_t'].iloc[0],'\t',"%6.2f"%t_rec['steal_avg_o'].iloc[0])
    print('Win %','\t\t',"%6.2f"%(100*t_rec['win_pct_t'].iloc[0]),'\t',"%6.2f"%(100*t_rec['win_pct_o'].iloc[0]))
    print('Off Rating','\t',"%6.2f"%t_rec['off_rating_t'].iloc[0],'\t',"%6.2f"%t_rec['off_rating_o'].iloc[0])
    print('SRS Rating','\t',"%6.2f"%t_rec['SRS_t'].iloc[0],'\t',"%6.2f"%t_rec['SRS_o'].iloc[0])
    #print('SOS Rating','\t',"%6.2f"%t_rec['SOS_t'].iloc[0],'\t',"%6.2f"%t_rec['SOS_o'].iloc[0])
    print('\n\nTop Conference= ', t_rec['top_conf'].iloc[0])
    print('Game Result= ', t_rec['game_result'].iloc[0], ' (One indicates win for ', team,')')
    print('Predicted Result= ', t_rec['predicted_result'].iloc[0])
    print('Predicted Probability= ',"%6.3f"%t_rec['pred_win_prob'].iloc[0])
    return

#### Read in regular season team statistics from SRCBB https://www.sports-reference.com/cbb/

In [7]:
# Note season indicates start of season, e.g. 2013 to 2014, the corresponding tournament season is 1 greater than this value
# The tournament season for season 2013 is 2014
summary_data = pd.read_csv('sr_summaries_kaggle_id.csv')
print(summary_data.shape)

#display rows with NaNs
summary_data[summary_data.isnull().any(axis=1)]


(2773, 22)


,season,School,team_id,Games,SRS,SOS,win_pct,pts_avg,opp_pts_avg,fg_pct,allow_fg_pct,ft_pct,allow_ft_att_avg,poss_avg,off_rebs_avg,allow_off_rebs_avg,def_rebs_avg,allow_def_rebs_avg,to_avg,steal_avg,off_rating,ft_att_avg
38,2009,Centenary (LA),NaN,29,-17.16,-5.08,0.276,70,80,0.416,0.49,0.733,25,80.641379,9,11,20,27,15,7,86.804071,21


In [8]:
#drop records with NaNs
summary_data.dropna(inplace=True)
print(summary_data.shape)


(2772, 22)


#### Read table of team names and associated team meta data from the Kaggle data set.

https://console.cloud.google.com/bigquery?project=bigqueryncaa&p=bigquery-public-data&d=ncaa_basketball&page=dataset

In [9]:
# Read table of team names and associated team meta data
teams = pd.read_csv('D1_teams.csv')
teams.drop(columns=['code_ncaa','school_ncaa','turner_name','league_name','league_alias','conf_alias',
                    'conf_id','division_name','division_alias','division_id',
                    'kaggle_team_id','venue_id'], inplace=True)
teams.info()
teams.set_index('id',inplace=True)
teams.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 2 columns):
id           351 non-null object
conf_name    351 non-null object
dtypes: object(2)
memory usage: 5.6+ KB


,conf_name
id,
fe406882-9f22-495e-9df6-ef357a6803c6,Ivy
ca478771-aa3d-4231-81e0-b70f519134fb,Ivy
5c7bf63f-bc39-43c5-9907-73b50b7a6b34,Ivy
d60357bd-1205-42e9-9092-d986a2843a34,Ivy
88ff8c00-958e-4ccf-a21d-77fab9e93692,Ivy


#### Read in the NCAA Men's Tournament results from the the Kaggle data set.

In [10]:
tourney_data = pd.read_csv('tournament_results.csv')
tourney_data.drop(columns=['days_from_epoch','day','num_ot','academic_year','win_region','win_alias','lose_region',
                           'lose_alias','lose_code_ncaa','win_school_ncaa','win_code_ncaa','win_name','lose_name',
                           'win_pts','win_kaggle_team_id','lose_school_ncaa','lose_kaggle_team_id','lose_pts'],inplace=True)
tourney_data['train_rec'] = 0
tourney_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2117 entries, 0 to 2116
Data columns (total 10 columns):
season          2117 non-null int64
round           2117 non-null int64
game_date       2117 non-null object
win_seed        2117 non-null int64
win_market      2117 non-null object
win_team_id     2117 non-null object
lose_seed       2117 non-null int64
lose_market     2117 non-null object
lose_team_id    2117 non-null object
train_rec       2117 non-null int64
dtypes: int64(5), object(5)
memory usage: 165.5+ KB


### Extract some historical data

In [11]:
game_data = tourney_data.join(teams, on='win_team_id',how='left')
game_data.rename(columns={'kaggle_team_id':'win_kaggle_team_id','conf_name':'win_conf_name'}, inplace=True)
game_data = game_data.join(teams,on='lose_team_id',how='left')
game_data.rename(columns={'kaggle_team_id':'lose_kaggle_team_id','conf_name':'lose_conf_name'}, inplace=True)
games_won_conf = game_data.groupby('win_conf_name').size().reset_index(name='count').sort_values(by=['count'], ascending=False)

games_won_conf['percent'] = 100 * games_won_conf['count']/games_won_conf['count'].sum()
games_won_conf['cum_percent'] = games_won_conf['percent'].cumsum()
games_won_conf.head()

,win_conf_name,count,percent,cum_percent
3,Atlantic Coast,431,20.358999,20.358999
9,Big Ten,313,14.785073,35.144072
24,Southeastern,269,12.706660,47.850732
5,Big 12,231,10.911667,58.762400
22,Pacific 12,197,9.305621,68.068021


Identify the conferences that have won 70% of all conference games

In [12]:
top_tournament_conferences_list = games_won_conf[games_won_conf['cum_percent']<= 70]['win_conf_name'].tolist()
top_tournament_conferences_list

['Atlantic Coast', 'Big Ten', 'Southeastern', 'Big 12', 'Pacific 12']

In [13]:
tourney_data.describe()['season']

count    2117.000000
mean     2001.193198
std         9.563883
min      1985.000000
25%      1993.000000
50%      2001.000000
75%      2010.000000
max      2017.000000
Name: season, dtype: float64

Note the tournament data is available for tournament years 1985 through 2017.

In [14]:
summary_data.describe()['season']

count    2772.00000
mean     2012.53355
std         2.28420
min      2009.00000
25%      2011.00000
50%      2013.00000
75%      2015.00000
max      2016.00000
Name: season, dtype: float64

The summary season data in this set goes back to 2009. 

So the intersection of season summary data with tournament data are regular seasons 2009 through 2016.

In [15]:
tourney_data = tourney_data[tourney_data['season'] >= 2010]
tourney_data.describe()['season']

count     533.000000
mean     2013.519700
std         2.284733
min      2010.000000
25%      2012.000000
50%      2014.000000
75%      2016.000000
max      2017.000000
Name: season, dtype: float64

In [16]:
tourney_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 10 to 2104
Data columns (total 10 columns):
season          533 non-null int64
round           533 non-null int64
game_date       533 non-null object
win_seed        533 non-null int64
win_market      533 non-null object
win_team_id     533 non-null object
lose_seed       533 non-null int64
lose_market     533 non-null object
lose_team_id    533 non-null object
train_rec       533 non-null int64
dtypes: int64(5), object(5)
memory usage: 45.8+ KB


In [17]:
tourney_data.head()

,season,round,game_date,win_seed,win_market,win_team_id,lose_seed,lose_market,lose_team_id,train_rec
10,2013,64,2013-03-22,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,0
17,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0
27,2013,64,2013-03-21,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,0
29,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0
30,2011,8,2011-03-27,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,0


In [18]:
# recode the tourney data so that the teams are not marked with  win/lose status and the game result is encoded as a binary
tourney_data['game_result'] = 1
tourney_data.game_result = tourney_data.game_result.astype(int)
tourney_data.rename(columns={"win_seed":"team_seed","win_market":"team","win_team_id":"team_id"}, inplace=True)
tourney_data.rename(columns={"lose_seed":"opp_team_seed","lose_market":"opp_team","lose_team_id":"opp_team_id"}, inplace=True)
tourney_data['start_season'] = tourney_data['season'] -1
tourney_data


,season,round,game_date,team_seed,team,team_id,opp_team_seed,opp_team,opp_team_id,train_rec,game_result,start_season
10,2013,64,2013-03-22,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,0,1,2012
17,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0,1,2010
27,2013,64,2013-03-21,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,0,1,2012
29,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0,1,2011
30,2011,8,2011-03-27,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,0,1,2010
31,2011,16,2011-03-25,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,10,Florida State,d3c8a767-b6fc-45d4-a5f2-18406f2993f1,0,1,2010
33,2011,32,2011-03-20,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,3,Purdue,12d7f888-675b-459f-9099-a38f771d8a95,0,1,2010
34,2011,64,2011-03-18,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,6,Georgetown,8736b67f-5924-400b-bf94-3bd804c36600,0,1,2010
35,2011,68,2011-03-16,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,11,USC,3a000455-de7c-4ca8-880e-abdce7f21da9,0,1,2010
36,2016,64,2016-03-18,10,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,7,Oregon State,532d3874-b4b3-4c5c-acc6-749a6db26c8f,0,1,2015


In [19]:
# We need to recode the game data so that we have representation of the losing game result class.
# Swap the team and opp team data on the odd rows and reverse the sense of the game result.

#create some temporary buffer columns
tourney_data['copy_team'] = tourney_data['team']
tourney_data['copy_team_seed'] = tourney_data['team_seed']
tourney_data['copy_team_id'] = tourney_data['team_id']

#swap the team and opp team data
tourney_data.loc[1::2,'team'] = tourney_data.loc[1::2,'opp_team']
tourney_data.loc[1::2,'opp_team'] = tourney_data.loc[1::2,'copy_team']
tourney_data.loc[1::2,'team_seed'] = tourney_data.loc[1::2,'opp_team_seed']
tourney_data.loc[1::2,'opp_team_seed'] = tourney_data.loc[1::2,'copy_team_seed']
tourney_data.loc[1::2,'team_id'] = tourney_data.loc[1::2,'opp_team_id']
tourney_data.loc[1::2,'opp_team_id'] = tourney_data.loc[1::2,'copy_team_id']

# flip the game result
tourney_data.loc[1::2,'game_result'] = 0

#drop the temporary columns
tourney_data.drop(columns=['copy_team','copy_team_seed','copy_team_id'],inplace=True)
tourney_data.head()

,season,round,game_date,team_seed,team,team_id,opp_team_seed,opp_team,opp_team_id,train_rec,game_result,start_season
10,2013,64,2013-03-22,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,0,0,2012
17,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0,1,2010
27,2013,64,2013-03-21,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2012
29,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0,1,2011
30,2011,8,2011-03-27,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2010


In [20]:

tourney_data = tourney_data.merge(summary_data, left_on=['start_season','team_id'], 
                                right_on=['season', 'team_id'],how='left',suffixes=('','_y'))

tourney_data.drop(columns=['season_y'],inplace=True)
tourney_data.describe()

,season,round,team_seed,opp_team_seed,train_rec,game_result,start_season,Games,SRS,SOS,win_pct,pts_avg,opp_pts_avg,fg_pct,allow_fg_pct,ft_pct,allow_ft_att_avg,poss_avg,off_rebs_avg,allow_off_rebs_avg,def_rebs_avg,allow_def_rebs_avg,to_avg,steal_avg,off_rating,ft_att_avg
count,533.000000,533.000000,533.000000,533.000000,533.0,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,2013.519700,44.675422,7.054409,7.358349,0.0,0.499062,2012.519700,35.816135,14.258555,5.737036,0.746683,73.409006,64.506567,0.458936,0.411435,0.708737,18.093809,77.084808,10.898687,10.204503,23.825516,21.983114,11.491557,6.251407,95.280433,21.060038
std,2.284733,22.471906,4.747263,4.693331,0.0,0.500469,2.284733,2.159078,7.612631,4.792314,0.100983,4.750710,4.666959,0.022228,0.021711,0.034613,2.824207,3.840933,1.748538,1.404015,2.242269,1.671774,1.542147,1.264251,4.975232,2.652874
min,2010.000000,2.000000,1.000000,1.000000,0.0,0.000000,2009.000000,30.000000,-13.370000,-10.310000,0.412000,59.000000,51.000000,0.394000,0.354000,0.603000,11.000000,63.960000,7.000000,7.000000,18.000000,18.000000,7.000000,3.000000,77.919708,14.000000
25%,2012.000000,32.000000,3.000000,3.000000,0.0,0.000000,2011.000000,34.000000,11.250000,4.010000,0.676000,70.000000,61.000000,0.443000,0.397000,0.685000,16.000000,74.916667,10.000000,9.000000,23.000000,21.000000,10.000000,5.000000,91.568956,19.000000
50%,2014.000000,64.000000,6.000000,7.000000,0.0,0.000000,2013.000000,36.000000,15.410000,7.540000,0.750000,73.000000,65.000000,0.458000,0.411000,0.707000,18.000000,76.751515,11.000000,10.000000,24.000000,22.000000,11.000000,6.000000,95.233182,21.000000
75%,2016.000000,64.000000,11.000000,11.000000,0.0,1.000000,2015.000000,37.000000,19.400000,9.040000,0.816000,77.000000,67.000000,0.473000,0.424000,0.731000,20.000000,79.517647,12.000000,11.000000,25.000000,23.000000,13.000000,7.000000,98.753538,23.000000
max,2017.000000,68.000000,16.000000,16.000000,0.0,1.000000,2016.000000,41.000000,28.720000,12.230000,0.974000,89.000000,80.000000,0.522000,0.495000,0.818000,28.000000,88.716667,16.000000,14.000000,31.000000,28.000000,16.000000,10.000000,107.899807,30.000000


In [21]:
tourney_data = tourney_data.merge(summary_data, left_on=['start_season','opp_team_id'], 
                                right_on=['season', 'team_id'], how='left',suffixes=('_t','_o'))
#tourney_data.info()

In [22]:
tourney_data.describe()

,season_t,round,team_seed,opp_team_seed,train_rec,game_result,start_season,Games_t,SRS_t,SOS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,season_o,Games_o,SRS_o,SOS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o
count,533.000000,533.000000,533.000000,533.000000,533.0,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,2013.519700,44.675422,7.054409,7.358349,0.0,0.499062,2012.519700,35.816135,14.258555,5.737036,0.746683,73.409006,64.506567,0.458936,0.411435,0.708737,18.093809,77.084808,10.898687,10.204503,23.825516,21.983114,11.491557,6.251407,95.280433,21.060038,2012.519700,35.754221,14.074897,5.630356,0.739158,73.393996,64.527205,0.458683,0.413379,0.709782,18.046904,76.985350,10.898687,10.091932,23.673546,21.866792,11.407129,6.258912,95.380277,20.938086
std,2.284733,22.471906,4.747263,4.693331,0.0,0.500469,2.284733,2.159078,7.612631,4.792314,0.100983,4.750710,4.666959,0.022228,0.021711,0.034613,2.824207,3.840933,1.748538,1.404015,2.242269,1.671774,1.542147,1.264251,4.975232,2.652874,2.284733,2.102989,7.612688,4.974119,0.094207,5.017874,4.579879,0.022908,0.020199,0.034179,2.798975,4.089687,1.829442,1.363112,2.316660,1.708857,1.517227,1.376679,5.032098,2.725871
min,2010.000000,2.000000,1.000000,1.000000,0.0,0.000000,2009.000000,30.000000,-13.370000,-10.310000,0.412000,59.000000,51.000000,0.394000,0.354000,0.603000,11.000000,63.960000,7.000000,7.000000,18.000000,18.000000,7.000000,3.000000,77.919708,14.000000,2009.000000,30.000000,-11.200000,-12.120000,0.412000,59.000000,51.000000,0.380000,0.354000,0.621000,11.000000,63.960000,5.000000,7.000000,18.000000,18.000000,7.000000,3.000000,76.996634,13.000000
25%,2012.000000,32.000000,3.000000,3.000000,0.0,0.000000,2011.000000,34.000000,11.250000,4.010000,0.676000,70.000000,61.000000,0.443000,0.397000,0.685000,16.000000,74.916667,10.000000,9.000000,23.000000,21.000000,10.000000,5.000000,91.568956,19.000000,2011.000000,34.000000,10.980000,3.800000,0.676000,70.000000,62.000000,0.443000,0.398000,0.686000,16.000000,74.771429,10.000000,9.000000,22.000000,21.000000,10.000000,5.000000,91.911493,19.000000
50%,2014.000000,64.000000,6.000000,7.000000,0.0,0.000000,2013.000000,36.000000,15.410000,7.540000,0.750000,73.000000,65.000000,0.458000,0.411000,0.707000,18.000000,76.751515,11.000000,10.000000,24.000000,22.000000,11.000000,6.000000,95.233182,21.000000,2013.000000,36.000000,15.410000,7.530000,0.743000,73.000000,64.000000,0.459000,0.414000,0.709000,18.000000,76.914286,11.000000,10.000000,24.000000,22.000000,11.000000,6.000000,95.286405,21.000000
75%,2016.000000,64.000000,11.000000,11.000000,0.0,1.000000,2015.000000,37.000000,19.400000,9.040000,0.816000,77.000000,67.000000,0.473000,0.424000,0.731000,20.000000,79.517647,12.000000,11.000000,25.000000,23.000000,13.000000,7.000000,98.753538,23.000000,2015.000000,37.000000,18.970000,9.070000,0.800000,77.000000,68.000000,0.475000,0.426000,0.733000,20.000000,79.520000,12.000000,11.000000,25.000000,23.000000,12.000000,7.000000,98.792943,23.000000
max,2017.000000,68.000000,16.000000,16.000000,0.0,1.000000,2016.000000,41.000000,28.720000,12.230000,0.974000,89.000000,80.000000,0.522000,0.495000,0.818000,28.000000,88.716667,16.000000,14.000000,31.000000,28.000000,16.000000,10.000000,107.899807,30.000000,2016.000000,41.00000

Incorporate additional features here.
Team conference, seeding deltas.

In [23]:
tourney_data = tourney_data.join(teams, on='team_id_t', how='left')


In [24]:
tourney_data = tourney_data.join(teams, on='opp_team_id', how='left', lsuffix='_t', rsuffix='_o')
tourney_data.head()

,season_t,round,game_date,team_seed,team,team_id_t,opp_team_seed,opp_team,opp_team_id,train_rec,game_result,start_season,School_t,Games_t,SRS_t,SOS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,season_o,School_o,team_id_o,Games_o,SRS_o,SOS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,conf_name_t,conf_name_o
0,2013,64,2013-03-22,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,0,0,2012,North Carolina State,35,14.32,7.17,0.686,77,70,0.496,0.419,0.681,17,78.434286,11,12,22,21,12,6,98.171354,23,2012,Temple,8598d1b6-106b-4083-a512-2e495729525a,34,9.52,5.20,0.706,72,67,0.428,0.438,0.723,18,77.494118,11,10,19,25,10,7,92.910278,20,Atlantic Coast,American Athletic
1,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0,1,2010,Temple,34,12.35,4.23,0.765,70,62,0.447,0.435,0.698,19,75.688235,11,11,24,22,10,6,92.484651,19,2010,Penn State,4aebd148-8119-4875-954c-66779867989b,34,11.98,11.45,0.559,63,62,0.439,0.434,0.724,21,68.341176,9,8,21,23,10,4,92.184541,16,American Athletic,Big Ten
2,2013,64,2013-03-21,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2012,Akron,33,8.11,0.18,0.788,71,62,0.455,0.395,0.643,19,78.884848,12,10,21,21,13,7,90.004610,20,2012,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,36,16.89,4.47,0.750,76,64,0.450,0.444,0.701,20,81.044444,13,11,18,23,11,11,93.775706,18,Mid-American,Atlantic 10
3,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0,1,2011,Virginia Commonwealth,36,8.79,0.63,0.806,68,59,0.410,0.424,0.685,18,76.305556,12,10,18,24,11,10,89.115399,20,2011,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,33,17.42,3.80,0.818,77,62,0.482,0.393,0.748,18,77.496970,10,9,24,21,12,6,99.358724,19,Atlantic 10,American Athletic
4,2011,8,2011-03-27,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2010,Kansas,38,24.10,7.62,0.921,81,64,0.508,0.380,0.679,18,80.126316,11,10,27,21,13,7,101.090384,22,2010,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,40,8.56,3.78,0.700,71,66,0.434,0.424,0.715,18,75.240000,10,10,21,24,11,8,94.364700,20,Big 12,Atlantic 10


#### Add a feature
top_conf = 1 if team is a top conference and opponent is not
top_conf = -1 if opp_team is a top_conference and team is not
top_conf = 0 otherwise

In [25]:
print(top_tournament_conferences_list)
tourney_data['top_conf'] = tourney_data.apply(lambda row: conf_compare(row.conf_name_t, row.conf_name_o),axis=1)
tourney_data.head(10)[['game_result','conf_name_t','conf_name_o','top_conf']]

['Atlantic Coast', 'Big Ten', 'Southeastern', 'Big 12', 'Pacific 12']


,game_result,conf_name_t,conf_name_o,top_conf
0,0,Atlantic Coast,American Athletic,1
1,1,American Athletic,Big Ten,-1
2,0,Mid-American,Atlantic 10,0
3,1,Atlantic 10,American Athletic,0
4,0,Big 12,Atlantic 10,1
5,1,Atlantic 10,Atlantic Coast,-1
6,0,Big Ten,Atlantic 10,1
7,1,Atlantic 10,Big East,0
8,0,Pacific 12,Atlantic 10,1
9,1,Atlantic 10,Pacific 12,-1


In [26]:
# Upsets occur less than 30% of the time when seeding deltas are more than 6
tourney_data['upset_seed_threshold'] = tourney_data.apply(lambda row: abs(row.team_seed - row.opp_team_seed) > 6, axis=1).astype(int)
tourney_data.head()


,season_t,round,game_date,team_seed,team,team_id_t,opp_team_seed,opp_team,opp_team_id,train_rec,game_result,start_season,School_t,Games_t,SRS_t,SOS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,season_o,School_o,team_id_o,Games_o,SRS_o,SOS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,conf_name_t,conf_name_o,top_conf,upset_seed_threshold
0,2013,64,2013-03-22,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,0,0,2012,North Carolina State,35,14.32,7.17,0.686,77,70,0.496,0.419,0.681,17,78.434286,11,12,22,21,12,6,98.171354,23,2012,Temple,8598d1b6-106b-4083-a512-2e495729525a,34,9.52,5.20,0.706,72,67,0.428,0.438,0.723,18,77.494118,11,10,19,25,10,7,92.910278,20,Atlantic Coast,American Athletic,1,0
1,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0,1,2010,Temple,34,12.35,4.23,0.765,70,62,0.447,0.435,0.698,19,75.688235,11,11,24,22,10,6,92.484651,19,2010,Penn State,4aebd148-8119-4875-954c-66779867989b,34,11.98,11.45,0.559,63,62,0.439,0.434,0.724,21,68.341176,9,8,21,23,10,4,92.184541,16,American Athletic,Big Ten,-1,0
2,2013,64,2013-03-21,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2012,Akron,33,8.11,0.18,0.788,71,62,0.455,0.395,0.643,19,78.884848,12,10,21,21,13,7,90.004610,20,2012,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,36,16.89,4.47,0.750,76,64,0.450,0.444,0.701,20,81.044444,13,11,18,23,11,11,93.775706,18,Mid-American,Atlantic 10,0,1
3,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0,1,2011,Virginia Commonwealth,36,8.79,0.63,0.806,68,59,0.410,0.424,0.685,18,76.305556,12,10,18,24,11,10,89.115399,20,2011,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,33,17.42,3.80,0.818,77,62,0.482,0.393,0.748,18,77.496970,10,9,24,21,12,6,99.358724,19,Atlantic 10,American Athletic,0,1
4,2011,8,2011-03-27,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2010,Kansas,38,24.10,7.62,0.921,81,64,0.508,0.380,0.679,18,80.126316,11,10,27,21,13,7,101.090384,22,2010,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,40,8.56,3.78,0.700,71,66,0.434,0.424,0.715,18,75.240000,10,10,21,24,11,8,94.364700,20,Big 12,Atlantic 10,1,1


In [27]:
tourney_data['srs_sqr_t'] = tourney_data['SRS_t'] * tourney_data['SRS_t'] 
tourney_data['srs_sqr_o'] = tourney_data['SRS_o'] * tourney_data['SRS_o'] 
tourney_data['off_rating_sqr_t'] = tourney_data['off_rating_t'] * tourney_data['off_rating_t'] 
tourney_data['off_rating_sqr_o'] = tourney_data['off_rating_o'] * tourney_data['off_rating_o'] 



In [28]:
numeric_feature_to_scale = ['pts_avg_t','opp_pts_avg_t',
                            'off_rebs_avg_t','allow_off_rebs_avg_t',
                            'def_rebs_avg_t','allow_def_rebs_avg_t',
                            'ft_att_avg_t','allow_ft_att_avg_t',
                            'to_avg_t','steal_avg_t',
                            'SRS_t',
                            'off_rating_t',
                            'pts_avg_o','opp_pts_avg_o',
                            'off_rebs_avg_o','allow_off_rebs_avg_o',
                            'def_rebs_avg_o','allow_def_rebs_avg_o',
                            'ft_att_avg_o','allow_ft_att_avg_o',
                            'to_avg_o','steal_avg_o',
                            'SRS_o',
                            'off_rating_o',
                             'srs_sqr_t','srs_sqr_o','off_rating_sqr_t','off_rating_sqr_o']

scaler =StandardScaler()
tourney_data[numeric_feature_to_scale] = scaler.fit_transform(tourney_data[numeric_feature_to_scale])
tourney_data.head()

,season_t,round,game_date,team_seed,team,team_id_t,opp_team_seed,opp_team,opp_team_id,train_rec,game_result,start_season,School_t,Games_t,SRS_t,SOS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,season_o,School_o,team_id_o,Games_o,SRS_o,SOS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,conf_name_t,conf_name_o,top_conf,upset_seed_threshold,srs_sqr_t,srs_sqr_o,off_rating_sqr_t,off_rating_sqr_o
0,2013,64,2013-03-22,8,North Carolina State,52df1e19-b142-4a76-a439-ad68455d0581,9,Temple,8598d1b6-106b-4083-a512-2e495729525a,0,0,2012,North Carolina State,35,0.008079,7.17,0.686,0.756596,1.178196,0.496,0.419,0.681,-0.387661,78.434286,0.057996,1.280032,-0.814903,-0.588619,0.330008,-0.199045,0.581608,0.731955,2012,Temple,8598d1b6-106b-4083-a512-2e495729525a,34,-0.598892,5.20,0.706,-0.278067,0.540433,0.428,0.438,0.723,-0.016773,77.494118,0.055431,-0.067506,-2.019259,1.835234,-0.928306,0.538822,-0.491310,-0.344465,Atlantic Coast,American Athletic,1,0,-0.309298,-0.944152,0.564366,-0.512594
1,2011,64,2011-03-17,7,Temple,8598d1b6-106b-4083-a512-2e495729525a,10,Penn State,4aebd148-8119-4875-954c-66779867989b,0,1,2010,Temple,34,-0.250945,4.23,0.765,-0.718252,-0.537592,0.447,0.435,0.698,0.321167,75.688235,0.057996,0.567120,0.077889,0.010110,-0.968104,-0.199045,-0.562468,-0.777260,2010,Penn State,4aebd148-8119-4875-954c-66779867989b,34,-0.275443,11.45,0.559,-2.073340,-0.552324,0.439,0.434,0.724,1.056054,68.341176,-1.038825,-1.536116,-1.155136,0.663761,-0.928306,-1.642383,-0.635667,-1.813265,American Athletic,Big Ten,-1,0,-0.599033,-0.642088,-0.580318,-0.653018
2,2013,64,2013-03-21,12,Akron,56fe0ab2-e4f0-47b9-8726-9ce23ebcde20,5,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2012,Akron,33,-0.808437,0.18,0.788,-0.507560,-0.537592,0.455,0.395,0.643,0.321167,78.884848,0.630440,-0.145793,-1.261299,-0.588619,0.979063,0.592680,-1.061414,-0.399956,2012,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,36,0.370138,4.47,0.750,0.519832,-0.115221,0.450,0.444,0.701,0.698445,81.044444,1.149688,0.666798,-2.451321,0.663761,-0.268590,3.447094,-0.319167,-1.078865,Mid-American,Atlantic 10,0,1,-1.077425,0.167481,-1.058146,-0.343703
3,2012,64,2012-03-15,12,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,5,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,0,1,2011,Virginia Commonwealth,36,-0.719028,0.63,0.806,-1.139638,-1.181013,0.410,0.424,0.685,-0.033247,76.305556,0.630440,-0.145793,-2.600486,1.207568,-0.319048,2.967856,-1.240309,-0.399956,2011,Wichita State,f38b58de-3273-4723-a78b-2f918e9aa2d5,33,0.439824,3.80,0.818,0.719307,-0.552324,0.482,0.393,0.748,-0.016773,77.496970,-0.491697,-0.801811,0.141048,-0.507711,0.391126,-0.188247,0.791357,-0.711665,Atlantic 10,American Athletic,0,1,-1.014051,0.271334,-1.226306,0.783481
4,2011,8,2011-03-27,1,Kansas,fae4855b-1b64-4b40-a632-9ed345e1e952,11,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,0,0,2010,Kansas,38,1.293993,7.62,0.921,1.599366,-0.108645,0.508,0.380,0.679,-0.033247,80.126316,0.057996,-0.145793,1.417076,-0.588619,0.979063,0.592680,1.168872,0.354651,2010,Virginia Commonwealth,c1c1e6df-a383-4fbd-ba7b-32d4f9ef9518,40,-0.725115,3.78,0.700,-0.477542,0.321882,0.434,0.424,0.715,-0.016773,75.240000,-0.491697,-0.067506,-1.155136,1.249497,-0.268590,1.265890,-0.202010,-0.344465,Big 12,Atlantic 10,1,1,1.762790,-1.043280,1.178463,-0.227863


In [29]:
feature_data = tourney_data.drop(columns=['round','game_date','team','team_id_t',
                                         'opp_team','opp_team_id','game_result','start_season',
                                         'season_o','team_id_o','Games_t','Games_o',
                                          'School_o','School_t','SOS_o','SOS_t',
                                         'conf_name_t','conf_name_o','train_rec']).copy()
feature_data.head()

,season_t,team_seed,opp_team_seed,SRS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,SRS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,top_conf,upset_seed_threshold,srs_sqr_t,srs_sqr_o,off_rating_sqr_t,off_rating_sqr_o
0,2013,8,9,0.008079,0.686,0.756596,1.178196,0.496,0.419,0.681,-0.387661,78.434286,0.057996,1.280032,-0.814903,-0.588619,0.330008,-0.199045,0.581608,0.731955,-0.598892,0.706,-0.278067,0.540433,0.428,0.438,0.723,-0.016773,77.494118,0.055431,-0.067506,-2.019259,1.835234,-0.928306,0.538822,-0.491310,-0.344465,1,0,-0.309298,-0.944152,0.564366,-0.512594
1,2011,7,10,-0.250945,0.765,-0.718252,-0.537592,0.447,0.435,0.698,0.321167,75.688235,0.057996,0.567120,0.077889,0.010110,-0.968104,-0.199045,-0.562468,-0.777260,-0.275443,0.559,-2.073340,-0.552324,0.439,0.434,0.724,1.056054,68.341176,-1.038825,-1.536116,-1.155136,0.663761,-0.928306,-1.642383,-0.635667,-1.813265,-1,0,-0.599033,-0.642088,-0.580318,-0.653018
2,2013,12,5,-0.808437,0.788,-0.507560,-0.537592,0.455,0.395,0.643,0.321167,78.884848,0.630440,-0.145793,-1.261299,-0.588619,0.979063,0.592680,-1.061414,-0.399956,0.370138,0.750,0.519832,-0.115221,0.450,0.444,0.701,0.698445,81.044444,1.149688,0.666798,-2.451321,0.663761,-0.268590,3.447094,-0.319167,-1.078865,0,1,-1.077425,0.167481,-1.058146,-0.343703
3,2012,12,5,-0.719028,0.806,-1.139638,-1.181013,0.410,0.424,0.685,-0.033247,76.305556,0.630440,-0.145793,-2.600486,1.207568,-0.319048,2.967856,-1.240309,-0.399956,0.439824,0.818,0.719307,-0.552324,0.482,0.393,0.748,-0.016773,77.496970,-0.491697,-0.801811,0.141048,-0.507711,0.391126,-0.188247,0.791357,-0.711665,0,1,-1.014051,0.271334,-1.226306,0.783481
4,2011,1,11,1.293993,0.921,1.599366,-0.108645,0.508,0.380,0.679,-0.033247,80.126316,0.057996,-0.145793,1.417076,-0.588619,0.979063,0.592680,1.168872,0.354651,-0.725115,0.700,-0.477542,0.321882,0.434,0.424,0.715,-0.016773,75.240000,-0.491697,-0.067506,-1.155136,1.249497,-0.268590,1.265890,-0.202010,-0.344465,1,1,1.762790,-1.043280,1.178463,-0.227863


In [31]:
X= feature_data[feature_data['season_t']>=2010]
y= tourney_data[tourney_data['season_t']>=2010]['game_result']
X= X.drop(columns=['season_t'])

#X= feature_data
#y= tourney_data['game_result']


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 5)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
X_train.head()

(426, 42)
(426,)
(107, 42)
(107,)


,team_seed,opp_team_seed,SRS_t,win_pct_t,pts_avg_t,opp_pts_avg_t,fg_pct_t,allow_fg_pct_t,ft_pct_t,allow_ft_att_avg_t,poss_avg_t,off_rebs_avg_t,allow_off_rebs_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,steal_avg_t,off_rating_t,ft_att_avg_t,SRS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,fg_pct_o,allow_fg_pct_o,ft_pct_o,allow_ft_att_avg_o,poss_avg_o,off_rebs_avg_o,allow_off_rebs_avg_o,def_rebs_avg_o,allow_def_rebs_avg_o,to_avg_o,steal_avg_o,off_rating_o,ft_att_avg_o,top_conf,upset_seed_threshold,srs_sqr_t,srs_sqr_o,off_rating_sqr_t,off_rating_sqr_o
140,3,14,0.364401,0.676,-0.086174,0.320302,0.499,0.417,0.711,-0.033247,75.200000,-1.086891,0.567120,-0.368507,-1.786077,0.979063,-0.199045,0.360932,-0.777260,-1.251048,0.595,0.120882,0.977536,0.430,0.418,0.719,0.698445,80.518919,0.055431,1.401103,0.573110,1.835234,0.391126,0.538822,-0.691496,0.757135,0,1,0.159212,-1.343002,0.338257,-0.707030
293,5,4,0.140878,0.757,-0.507560,-0.323119,0.471,0.406,0.685,-0.387661,78.043243,1.202883,-0.145793,0.524285,-1.786077,0.979063,-0.199045,-0.866145,-0.399956,0.483214,0.727,1.118256,0.758985,0.471,0.394,0.729,-0.016773,82.139394,0.602560,2.869712,0.573110,0.663761,-0.268590,0.538822,0.158669,-0.344465,0,0,-0.144156,0.337619,-0.872689,0.133314
368,13,4,-1.397485,0.618,-1.350330,-0.537592,0.461,0.394,0.684,0.321167,72.452941,-1.086891,0.567120,0.524285,0.010110,-0.319048,-0.990771,-0.564648,-2.286475,0.531862,0.750,-1.075966,-1.207979,0.429,0.392,0.668,-0.016773,77.022222,0.602560,0.666798,0.141048,0.663761,-0.268590,0.538822,-1.411088,0.389935,-1,1,-1.367465,0.413417,-0.582434,-1.388465
53,3,11,0.551108,0.757,0.545903,0.320302,0.449,0.407,0.672,-0.033247,80.497297,0.630440,-0.145793,1.417076,1.207568,-0.319048,0.592680,-0.174492,0.354651,-0.696189,0.639,-1.075966,-0.333773,0.455,0.423,0.757,-1.804820,68.727778,-3.227338,-0.801811,0.141048,1.835234,-1.588022,-0.915315,0.708281,-1.446065,1,1,0.437045,-1.021493,-0.199789,0.696904
33,3,1,0.860096,0.846,0.967289,0.105828,0.481,0.403,0.712,-1.096490,77.917949,-0.514447,-0.145793,0.970681,0.010110,-0.319048,-0.199045,0.970689,-0.399956,1.243186,0.861,1.916156,1.414639,0.488,0.420,0.678,-0.016773,82.350000,0.055431,0.666798,1.005171,0.663761,0.391126,-0.188247,1.075917,0.022735,0,0,0.945695,1.700296,0.969214,1.082798


In [33]:
# mark the records used for training
tourney_data['train_rec'] = 0
tourney_data.loc[X_train.index,'train_rec']= 1
#tourney_data.head(30)

In [34]:
logreg = LogisticRegression()

result = logreg.fit(X_train,y_train)

print("Coeffs ",logreg.coef_)
print("Intercept ", logreg.intercept_)

Coeffs  [[ 0.08373137 -0.06693462  0.82934159  1.2522917   0.34293633 -0.20906029
  -0.12966117 -0.06080475 -0.35262557 -0.0751878   0.07552622 -0.13961919
  -0.07011049 -0.30351065 -0.08573423 -0.08412193 -0.12212707 -0.41916658
  -0.23503192 -0.89145825 -1.81625908  1.40162172 -0.6094567  -0.00842099
   0.13602882 -0.25304516  0.16269373 -0.065125   -0.29050786 -0.11594003
   0.14209621  0.0881142  -0.03061794 -0.2676942  -0.44462304  0.02697143
  -0.22188037 -0.0182804   0.76452509 -0.90846315 -0.13224097 -0.35284336]]
Intercept  [-0.08653086]


In [37]:
display_important_features_regression(logreg, X_train)

Coefficient Values
win_pct_o   -1.8162590815826913
pts_avg_o   1.4016217158097195
win_pct_t   1.25229169747292
srs_sqr_o   -0.9084631543542863
SRS_o   -0.8914582499397624
SRS_t   0.8293415910058354
srs_sqr_t   0.7645250945554546
opp_pts_avg_o   -0.6094566986731526
off_rating_o   -0.44462304323434815
off_rating_t   -0.4191665806189975
off_rating_sqr_o   -0.352843356299718
ft_pct_t   -0.3526255747243186
pts_avg_t   0.3429363348158932
def_rebs_avg_t   -0.3035106503880851
off_rebs_avg_o   -0.2905078610936012
steal_avg_o   -0.2676941963251784
ft_pct_o   -0.25304516453449766
ft_att_avg_t   -0.23503192083901644
top_conf   -0.2218803657834573
opp_pts_avg_t   -0.20906029294541897
allow_ft_att_avg_o   0.16269372979067281
def_rebs_avg_o   0.14209620833631814
off_rebs_avg_t   -0.1396191941376779
allow_fg_pct_o   0.13602881780028503
off_rating_sqr_t   -0.1322409710785797
fg_pct_t   -0.12966116579090015
steal_avg_t   -0.12212706908102967
allow_off_rebs_avg_o   -0.11594002791736191
allow_def_rebs_avg

In [ ]:
y_pred = logreg.predict(X_test)
prediction_probabilities = logreg.predict_proba(X_test)
print("LR model prediction accuracy is %2.2f" % metrics.accuracy_score(y_test, y_pred))

In [ ]:
print("Log loss= ",log_loss(y_test,logreg.predict_proba(X_test)))

In [ ]:
#X_2010 = feature_data[feature_data['season_t']==2010]
#y_2010 = tourney_data[tourney_data['season_t']==2010]['game_result']
#X_2010 = X_2010.drop(columns=['season_t'])
#print("2010 y sizes= ", y_2010.shape)
#print("2010 x sizes= ", X_2010.shape)

#y_pred_2010 = logreg.predict(X_2010)
#print("LR 2010 model prediction accuracy is %2.2f" % metrics.accuracy_score(y_2010, y_pred_2010))

In [ ]:
#cnf_matrix_2010 = metrics.confusion_matrix(y_2010, y_pred_2010)
#cnf_matrix_2010
#class_names=[0,1] # name  of classes
#fig, ax = plt.subplots()
#tick_marks = np.arange(len(class_names))
#plt.xticks(tick_marks, class_names)
#plt.yticks(tick_marks, class_names)
# create heatmap
#sns.heatmap(pd.DataFrame(cnf_matrix_2010), annot=True, cmap="YlGnBu" ,fmt='g')
#ax.xaxis.set_label_position("top")
#plt.tight_layout()
#plt.title('Confusion matrix', y=1.1)
#plt.ylabel('Actual label')
#plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
print("Log loss= ",log_loss(y_test, prediction_probabilities))

In [ ]:
# Retrieve the non-normalized game stats
prediction_probabilities[:,1]
pred_probs = pd.Series(prediction_probabilities[:,1], index=X_test.index)
predictions = pd.Series(y_pred, index=y_test.index)
test_games = tourney_data[tourney_data.index.isin(X_test.index)].copy()

test_games[numeric_feature_to_scale] = scaler.inverse_transform(test_games[numeric_feature_to_scale])
test_games['predicted_result'] = predictions
test_games['pred_win_prob'] = pred_probs

test_games.head()

In [ ]:
missed_predictions = test_games[test_games['game_result'] != 
                                test_games['predicted_result']].sort_values(by='pred_win_prob', ascending=False)
missed_predictions[['predicted_result','game_result','pred_win_prob','team','team_seed','conf_name_t', 'opp_team','opp_team_seed', 
                    'conf_name_o','top_conf','season_t','round']]

print("Total missed predictions= ", missed_predictions.shape[0])

missed_predictions

### Missed Prediction Analysis

#### Type I Errors

In [ ]:
type_1_cond = (missed_predictions['predicted_result'] == 1) &  (missed_predictions['game_result'] == 0)
t1_errors = missed_predictions[type_1_cond]
t1_errors

In [ ]:
for index, row in t1_errors.iterrows():
    print_game_info(test_games, row['season_t'], row['round'], row['team'])
    print("------\n")

In [ ]:
y_train.shape

#### Type II Errors

#### Missed predictions where higher seed was upset

In [ ]:
f_list=['season_t','round', 'team_seed','SRS_t','team','opp_team','opp_team_seed','SRS_o','game_result','predicted_result']

cond_1 = ((missed_predictions['team_seed'] > missed_predictions['opp_team_seed']) & 
          (missed_predictions['game_result'] == 1))

cond_2 = ((missed_predictions['team_seed'] < missed_predictions['opp_team_seed']) &  
          (missed_predictions['game_result'] == 0))

upset_seeds = missed_predictions[cond_2 | cond_1][f_list]
print("Total missed predictions with seeding upset= ", upset_seeds.shape[0])
upset_seeds

In [ ]:
for index, row in upset_seeds.iterrows():
    print_game_info(test_games, row['season_t'], row['round'], row['team'])
    print("------\n")



In [ ]:
tourney_data[((tourney_data['team']=='Missouri') & (tourney_data['train_rec']==1))]

#### Missed predictions where Lower SRS Won Game

In [ ]:
cond_srs_1 = ((missed_predictions['SRS_t'] > missed_predictions['SRS_o']) & 
          (missed_predictions['game_result'] == 0))

cond_srs_2 = ((missed_predictions['SRS_t'] < missed_predictions['SRS_o']) & 
          (missed_predictions['game_result'] == 1))

upset_srs = missed_predictions[cond_srs_1 | cond_srs_2][f_list]
print("Total missed predictions with SRS upset= ", upset_srs.shape[0])
upset_srs

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(LogisticRegression(), 
                                                        X, 
                                                        y,
                                                        # Number of folds in cross-validation
                                                        cv=10,
                                                        # Evaluation metric
                                                        scoring='accuracy',
                                                        # Use all computer cores
                                                        n_jobs=-1, 
                                                        # 50 different sizes of the training set
                                                        train_sizes=np.linspace(0.01, 1.0, 50))

In [ ]:
# Create means and standard deviations of training set scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

### Cross validation error

In [ ]:

scores = cross_val_score(LogisticRegression(), X,y, cv=10, scoring='accuracy')
print(scores.mean())

### Recursive Feature Elimination

In [36]:
from sklearn.feature_selection import RFE
rfe = RFE(logreg,20)
fit = rfe.fit(X,y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

Num Features: 20
Selected Features: [False False  True  True  True False False False False False False False
 False  True  True  True False  True  True  True  True  True  True False
 False  True False False  True False False False False  True  True False
  True False  True  True False  True]
Feature Ranking: [ 8 14  1  1  1 11 18 17  7 15 19  4  9  1  1  1 22  1  1  1  1  1  1 13
 12  1  5 20  1 10  2  6 16  1  1 23  1 21  1  1  3  1]


In [40]:
X_train.iloc[:,fit.support_]

,SRS_t,win_pct_t,pts_avg_t,def_rebs_avg_t,allow_def_rebs_avg_t,to_avg_t,off_rating_t,ft_att_avg_t,SRS_o,win_pct_o,pts_avg_o,opp_pts_avg_o,ft_pct_o,off_rebs_avg_o,steal_avg_o,off_rating_o,top_conf,srs_sqr_t,srs_sqr_o,off_rating_sqr_o
140,0.364401,0.676,-0.086174,-0.368507,-1.786077,0.979063,0.360932,-0.777260,-1.251048,0.595,0.120882,0.977536,0.719,0.055431,0.538822,-0.691496,0,0.159212,-1.343002,-0.707030
293,0.140878,0.757,-0.507560,0.524285,-1.786077,0.979063,-0.866145,-0.399956,0.483214,0.727,1.118256,0.758985,0.729,0.602560,0.538822,0.158669,0,-0.144156,0.337619,0.133314
368,-1.397485,0.618,-1.350330,0.524285,0.010110,-0.319048,-0.564648,-2.286475,0.531862,0.750,-1.075966,-1.207979,0.668,0.602560,0.538822,-1.411088,-1,-1.367465,0.413417,-1.388465
53,0.551108,0.757,0.545903,1.417076,1.207568,-0.319048,-0.174492,0.354651,-0.696189,0.639,-1.075966,-0.333773,0.757,-3.227338,-0.915315,0.708281,1,0.437045,-1.021493,0.696904
33,0.860096,0.846,0.967289,0.970681,0.010110,-0.319048,0.970689,-0.399956,1.243186,0.861,1.916156,1.414639,0.678,0.055431,-0.188247,1.075917,0,0.945695,1.700296,1.082798
342,-2.317874,0.606,-0.086174,-0.814903,-0.588619,-0.319048,1.155362,-1.909171,1.190593,0.917,0.519832,-0.989427,0.731,-0.491697,0.538822,1.494925,0,-1.377501,1.593703,1.531326
153,0.485366,0.780,-0.296867,0.524285,-0.588619,-0.319048,-0.746692,-0.399956,-1.536366,0.735,-0.677017,-0.333773,0.788,-2.133081,-1.642383,0.053944,0,0.336679,-1.429136,0.027736
392,-0.633563,0.656,-0.086174,-1.707694,0.010110,0.330008,0.301059,-1.154564,-0.517372,0.706,-0.278067,0.321882,0.737,-1.038825,0.538822,0.026414,0,-0.948706,-0.874538,0.000078
371,1.079674,0.735,0.756596,0.970681,0.608839,-0.319048,0.038297,-0.399956,-2.225337,0.571,-0.876491,0.540433,0.714,-0.491697,-1.642383,-0.530461,1,1.344183,-1.415369,-0.550787
84,-1.511876,0.703,-0.296867,-0.814903,0.608839,1.628119,-1.009581,-0.777260,1.286575,0.784,-0.477542,-2.300736,0.677,-0.491697,0.538822,0.670667,-1,-1.398122,1.789612,0.657825
